# El Problema
Se desea construir un modelo predictivo que dados ciertos datos de un estudiante, como el tipo de red que tiene,el dispositivo usa para tomar la clase o su situacion financiera, pueda predecir el nivel de adaptabilidad de dicho estudiante a sus clases online.

Como se puede ver este es un problema de clasificacion, para clasificar si el estudiante tiene un nivel de adaptabilidad de Bajo, Moderado o Alto

# Preparamos los datos

Se preparara el dataset para que la red neuronal pueda ser entrenada mediante limpieza de datos y hot encoding, en este caso no se encontraron valores nulos por lo que solo seria nesesario hacer el hot encoding

In [264]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

datasetDirecction = "./training-ds.csv"
data_students = pd.read_csv(datasetDirecction)
data_students

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase,Nivel de Adaptación
0,3G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Media,Masculino,03-Jun,Bajo
1,3G,No,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Femenino,01-Mar,Moderado
2,3G,Si,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Mala,Masculino,01-Mar,Bajo
3,3G,Si,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Masculino,01-Mar,Moderado
4,4G,Si,Universidad,Si,Privado,21-25,Computadora,Wifi,Buena,Masculino,01-Mar,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...
959,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,01-Mar,Moderado
960,3G,No,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Femenino,01-Mar,Moderado
961,4G,No,Universidad,Si,Público,21-25,Smartphone,Compra Megas,Media,Masculino,0,Bajo
962,3G,No,Universidad,Si,Privado,21-25,Smartphone,Compra Megas,Media,Masculino,01-Mar,Moderado


In [265]:
labels_map = {
    0: "Alto",
    1: "Bajo",
    2: "Moderado"
}
data_students_training = pd.get_dummies(data_students, columns=["Tipo de Red", "Estudiante de Tecnología", "Nivel de Educación", "Vive en Ciudad", "Tipo de Instituto", "Edad", "Dispositivo", "Tipo de Internet", "Situación Financiera", "Género", "Duración de la Clase"])
data_students_training["Nivel de Adaptación"] = data_students_training["Nivel de Adaptación"].astype('category').cat.codes
data_students_training

,Nivel de Adaptación,Tipo de Red_2G,Tipo de Red_3G,Tipo de Red_4G,Estudiante de Tecnología_No,Estudiante de Tecnología_Si,Nivel de Educación_Colegio,Nivel de Educación_Escuela,Nivel de Educación_Universidad,Vive en Ciudad_No,...,Tipo de Internet_Compra Megas,Tipo de Internet_Wifi,Situación Financiera_Buena,Situación Financiera_Mala,Situación Financiera_Media,Género_Femenino,Género_Masculino,Duración de la Clase_0,Duración de la Clase_01-Mar,Duración de la Clase_03-Jun
0,1,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,1,0,0,1
1,2,0,1,0,1,0,0,1,0,0,...,1,0,0,0,1,1,0,0,1,0
2,1,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,1,0
3,2,0,1,0,0,1,0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
4,0,0,0,1,0,1,0,0,1,0,...,0,1,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,2,0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,1,0
960,2,0,1,0,1,0,0,1,0,0,...,1,0,0,0,1,1,0,0,1,0
961,1,0,0,1,1,0,0,0,1,0,...,1,0,0,0,1,0,1,1,0,0
962,2,0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,1,0


Ahora separamos los datos entre los datos de entrada(x) y el target(y) ademas de separarlos para el training y el test

In [266]:
dataset_x = data_students_training.iloc[0:,1:].to_numpy()
dataset_y = data_students_training.iloc[0:,0].to_numpy()


x_train, x_test, y_train, y_test = train_test_split(dataset_x, dataset_y, train_size=0.8)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))


# Armamos la arquitectura de la red neuronal
Ahora construimos nuestra arquitectura de la red neuronal usando tensorflow, esta arquitectura deberia tener 31 entradas y 3 salidas, para esta prueba utlizaremos una capa oculta de 32 neuronas

In [267]:
class NeuralNetwork(tf.keras.Model):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.sequence = tf.keras.Sequential([
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(3)
    ])

  def call(self, x: tf.Tensor) -> tf.Tensor:
    y_prime = self.sequence(x)
    return y_prime
model = NeuralNetwork()
model.build((1,31))
model.summary()

Model: "neural_network_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_26 (Sequential)   (1, 3)                    1123      
Total params: 1,123
Trainable params: 1,123
Non-trainable params: 0
_________________________________________________________________


# Entrenamos la red neuronal
Con los datos supervisados de training entrenamos la red neuronal con los siguientes parametros

In [268]:
learning_rate = 0.1
batch_size = 64
epochs = 10
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ["accuracy"]

train_dataset_batched = train_dataset.batch(batch_size).shuffle(500)
test_dataset_batched = test_dataset.batch(batch_size).shuffle(500)

model.compile(optimizer, loss_function, metrics)
print('\nFitting:')
model.fit(train_dataset_batched, epochs=epochs)



Fitting:
Epoch 1/10
13/13 [==============================] - 0s 829us/step - loss: 0.9537 - accuracy: 0.5553
Epoch 2/10
13/13 [==============================] - 0s 997us/step - loss: 0.8796 - accuracy: 0.5987
Epoch 3/10
13/13 [==============================] - 0s 831us/step - loss: 0.8326 - accuracy: 0.6220
Epoch 4/10
13/13 [==============================] - 0s 748us/step - loss: 0.7968 - accuracy: 0.6559
Epoch 5/10
13/13 [==============================] - 0s 665us/step - loss: 0.7838 - accuracy: 0.6648
Epoch 6/10
13/13 [==============================] - 0s 831us/step - loss: 0.7445 - accuracy: 0.6643
Epoch 7/10
13/13 [==============================] - 0s 665us/step - loss: 0.7650 - accuracy: 0.6599
Epoch 8/10
13/13 [==============================] - 0s 665us/step - loss: 0.7756 - accuracy: 0.6398
Epoch 9/10
13/13 [==============================] - 0s 831us/step - loss: 0.7303 - accuracy: 0.6749
Epoch 10/10
13/13 [==============================] - 0s 748us/step - loss: 0.7338 - accura

# Evaluando el modelo
Despues de entrenar el modelo y ver su accuracy, evaluamosel modelo con el dataset para test

In [269]:
print('\nEvaluating:')
test_loss, test_accuracy = model.evaluate(test_dataset_batched)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:
4/4 [==============================] - 0s 1ms/step - loss: 0.7736 - accuracy: 0.6269

Test accuracy: 62.7%, test loss: 0.773563


In [270]:
datasetDirecction = "./test-ds.csv"
data_students = pd.read_csv(datasetDirecction)
data_students

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase
0,3G,No,Colegio,No,Público,21-25,Smartphone,Compra Megas,Mala,Masculino,01-Mar
1,4G,No,Colegio,No,Privado,16-20,Smartphone,Wifi,Media,Femenino,01-Mar
2,4G,No,Universidad,Si,Público,21-25,Smartphone,Compra Megas,Media,Masculino,0
3,3G,Si,Universidad,No,Público,21-25,Smartphone,Compra Megas,Media,Masculino,01-Mar
4,3G,No,Escuela,No,Público,06-Oct,Smartphone,Compra Megas,Media,Femenino,0
...,...,...,...,...,...,...,...,...,...,...,...
236,4G,No,Escuela,Si,Privado,01-May,Smartphone,Wifi,Media,Femenino,01-Mar
237,4G,No,Escuela,Si,Privado,01-May,Smartphone,Compra Megas,Media,Femenino,01-Mar
238,4G,No,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Masculino,01-Mar
239,4G,No,Escuela,Si,Público,16-20,Smartphone,Wifi,Media,Masculino,01-Mar


## Haciendo una prediccion
Una vez se tenga un modelo entrenado y probado con un buen accuracy podemos empezar a hacer predicciones con otros datos, para esta practica se utilizara el test-ds.csv para predecir el nivel de adaptacion

# Preparacion de datos

In [271]:
datasetDirecction = "./test-ds.csv"
data_students_test = pd.read_csv(datasetDirecction)
data_students_test

,Tipo de Red,Estudiante de Tecnología,Nivel de Educación,Vive en Ciudad,Tipo de Instituto,Edad,Dispositivo,Tipo de Internet,Situación Financiera,Género,Duración de la Clase
0,3G,No,Colegio,No,Público,21-25,Smartphone,Compra Megas,Mala,Masculino,01-Mar
1,4G,No,Colegio,No,Privado,16-20,Smartphone,Wifi,Media,Femenino,01-Mar
2,4G,No,Universidad,Si,Público,21-25,Smartphone,Compra Megas,Media,Masculino,0
3,3G,Si,Universidad,No,Público,21-25,Smartphone,Compra Megas,Media,Masculino,01-Mar
4,3G,No,Escuela,No,Público,06-Oct,Smartphone,Compra Megas,Media,Femenino,0
...,...,...,...,...,...,...,...,...,...,...,...
236,4G,No,Escuela,Si,Privado,01-May,Smartphone,Wifi,Media,Femenino,01-Mar
237,4G,No,Escuela,Si,Privado,01-May,Smartphone,Compra Megas,Media,Femenino,01-Mar
238,4G,No,Escuela,Si,Privado,Nov-15,Smartphone,Compra Megas,Media,Masculino,01-Mar
239,4G,No,Escuela,Si,Público,16-20,Smartphone,Wifi,Media,Masculino,01-Mar


In [272]:
data_students_test = pd.get_dummies(data_students_test, columns=None)
data_students_test

,Tipo de Red_2G,Tipo de Red_3G,Tipo de Red_4G,Estudiante de Tecnología_No,Estudiante de Tecnología_Si,Nivel de Educación_Colegio,Nivel de Educación_Escuela,Nivel de Educación_Universidad,Vive en Ciudad_No,Vive en Ciudad_Si,...,Tipo de Internet_Compra Megas,Tipo de Internet_Wifi,Situación Financiera_Buena,Situación Financiera_Mala,Situación Financiera_Media,Género_Femenino,Género_Masculino,Duración de la Clase_0,Duración de la Clase_01-Mar,Duración de la Clase_03-Jun
0,0,1,0,1,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,1,0
1,0,0,1,1,0,1,0,0,1,0,...,0,1,0,0,1,1,0,0,1,0
2,0,0,1,1,0,0,0,1,0,1,...,1,0,0,0,1,0,1,1,0,0
3,0,1,0,0,1,0,0,1,1,0,...,1,0,0,0,1,0,1,0,1,0
4,0,1,0,1,0,0,1,0,1,0,...,1,0,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,0,1,1,0,0,1,0,0,1,...,0,1,0,0,1,1,0,0,1,0
237,0,0,1,1,0,0,1,0,0,1,...,1,0,0,0,1,1,0,0,1,0
238,0,0,1,1,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,1,0
239,0,0,1,1,0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,1,0


In [273]:
dataset_test_x = data_students_test.iloc[0:].to_numpy()
predicted_names = []
for row in dataset_test_x:
    predicted_vector = model.predict(np.array([row]))
    predicted_index = np.argmax(predicted_vector)
    predicted_name = labels_map[predicted_index]
    predicted_names.append(predicted_name)

predicted_data = pd.DataFrame({'Nivel de Adaptación': np.array(predicted_names)})
predicted_data

,Nivel de Adaptación
0,Moderado
1,Moderado
2,Bajo
3,Bajo
4,Bajo
...,...
236,Moderado
237,Moderado
238,Moderado
239,Bajo


In [274]:
predicted_data.to_csv('predicted-ds.csv')

Dudas
- Para el nivel de educacion colegio y escuela son lo mismo
- las fechas en las edades

# Fuentes
- https://docs.microsoft.com/en-us/learn/modules/intro-machine-learning-keras/4-train
- https://docs.microsoft.com/en-us/learn/modules/intro-machine-learning-keras/2-data?source=learn
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html